In [1]:
import selenium
import bs4
import pandas as pd
import time

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium import webdriver

In [5]:


driver = webdriver.Chrome(r"E:\Scrap\chromedriver")
driver.get("https://www.daraz.pk/smartphones/?spm=a2a0e.home.cate_1.1.37c04937G4As9w")

delay = 5 # seconds
content = None


title = list()
image =  list()
link = list()
price = list()
rating = list()
no_of_review = list()
prod_uniqid = list()
prod_category = list() #Mobiles
driver.implicitly_wait(20)

for idx, links in enumerate(driver.find_elements_by_xpath('//div[@class="cRjKsc"]/a')):
    image.append(links.find_element_by_class_name("c1ZEkM ").get_attribute("src"))
    link.append(links.get_attribute("href"))
    title.append(links.find_element_by_class_name("c1ZEkM ").get_attribute("alt"))

for idx, links in enumerate(driver.find_elements_by_class_name('c2prKC')):
    prod_uniqid.append(links.get_attribute("data-item-id"))
                       
for idx, links in enumerate(driver.find_elements_by_xpath('//*[@id="J_breadcrumb"]/li[3]/span/span')):
     prod_category.append(links.text)
                
for idx, links in enumerate(driver.find_elements_by_xpath('//div[@class="c3gUW0"]/span')):
    price.append(links.text)
                       
for idx, links in enumerate(driver.find_elements_by_xpath('//div[@class="c15YQ9"]')):
    try:
        no_of_review.append(links.find_element_by_class_name("c3XbGJ").text)
    except NoSuchElementException:
        no_of_review.append(None)
page_soup = bs4.BeautifulSoup(driver.page_source,"lxml")

scirpt_data = str(page_soup).split('window.pageData')[1].split(r"}}}")[0]
for review in scirpt_data.split('ratingScore":'):
    rating.append(review.split(",")[0].replace('"',""))
rating.pop(0)

d = len(link) - len(rating)
for i in range(d):
    image.pop()
    link.pop()
    title.pop()
    price.pop()
    no_of_review.pop()
    
    
print(len(image))
var1=(len(image))
brand = prod_category[0]
for i in range(var1-1):
    prod_category.append(brand)
    
print(len(link))
print(len(title))
print(len(price))
print(len(no_of_review))
print(len(rating))
print(len(prod_uniqid))
print(len(prod_category))

<ipython-input-5-cc813514b72e>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"E:\Scrap\chromedriver")
<ipython-input-5-cc813514b72e>:18: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  for idx, links in enumerate(driver.find_elements_by_xpath('//div[@class="cRjKsc"]/a')):
<ipython-input-5-cc813514b72e>:23: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  for idx, links in enumerate(driver.find_elements_by_class_name('c2prKC')):
<ipython-input-5-cc813514b72e>:26: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  for idx, links in enumerate(driver.find_elements_by_xpath('//*[@id="J_breadcrumb"]/li[3]/span/span')):
<ipython-input-5-cc813514b72e>:29: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  for idx, link

40
40
40
40
40
40
40
40


In [6]:
df = pd.DataFrame({'Links':link, 'Images':image, "Title":title, "SKU_ID":prod_uniqid, "Product_Category":prod_category, "Price":price, "No of Review":no_of_review, "Rating":rating})
df.to_csv(brand + '.csv', index=False, mode='a')#,header=False

In [7]:
from urllib.parse import urlparse, parse_qs
import pandas as pd
import os
import requests as requests

In [8]:
file_name = 'Mobiles.csv'
folder_name = file_name[0: file_name.index('.')]


def make_folder(name):
    if not os.path.isdir(name):
        os.mkdir(name)


def get_df_from_csv(csv_name):
    dfr = pd.read_csv(csv_name)
    return dfr


def get_extension(name: str)-> str:
    idx = name.rfind('.')
    return name[idx:]


def download_image(row):
    url = row[0]
    extension = get_extension(urlparse(url).path)
    img_name = str(row[1]) + extension

    with open(folder_name + '/' + img_name, 'wb') as f:
        response = requests.get(url)
        f.write(response.content)


df = get_df_from_csv(file_name)
df = df.drop(columns=['Links', 'Title', 'Price', 'Product_Category','No of Review', 'Rating',] )
not_nan_images = df[~df['Images'].isna()]
make_folder(folder_name)
not_nan_images.apply(download_image, axis=1)


0    None
1    None
2    None
dtype: object